In [1]:
import tensorflow as tf
import requests
import pandas as pd
import numpy as np

import json
import math

2023-07-08 17:50:33.664124: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-07-08 17:50:35.746391: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-07-08 17:50:35.746450: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2023-07-08 17:50:43.029938: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-

In [8]:
url = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExperimentoPreRedes/proximoExperimentoParaPontuar"

r = requests.get(url)
j = r.json()
dfExperimento = pd.DataFrame.from_dict(j)
dfExperimento.head()

,nome,nomeArquivoRede,mediaY0,mediaY1,mediaResultadoNeutro,mediaResultadoPositivo,mediaResultadoNegativo,mediaTotal,desvioY0,desvioY1,desvioResultadoNeutro,desvioResultadoPositivo,desvioResultadoNegativo,desvioTotal,qtdeY0,qtdeY1,id,amostraDadoSinteticoId,versaoPreRedeId
0,Inicial1,ExperimentoSintetico_Prometeu_v0_aInicial,0.07524,0.82131,0.13081,0.82131,0.00116,0.29906,0.235101,0.000026,0.299205,0.000026,0,0.394436,6720,2880,1,1,15


In [3]:
def trataLinhaX(linha):
    return np.fromstring(linha['campoX'], dtype=float, sep=',')

In [4]:
def executaPrevisao():
    dfPrevisao = Y
    result = modelo.predict(X)
    dfPrevisao['score'] = result
    dfPrevisao['experimentoId'] = str(experimentoCorrente['id'])
    dfPrevisao.drop('campoY', inplace=True, axis=1)
    urlPrevisao = 'http://vps-40d69db1.vps.ovh.ca:23004/api/ExperimentoPreRedeResultados/recebePrevisaoTeste'
    print(urlPrevisao)
    
    # Definir o tamanho máximo de cada parte
    max_chunk_size = 20

    # Calcular o número de partes necessárias
    num_chunks = math.ceil(len(dfPrevisao) / max_chunk_size)

    # Dividir o DataFrame em partes menores e enviar ao servidor
    for i in range(num_chunks):
        # Calcular o índice inicial e final para cada parte
        start_index = i * max_chunk_size
        end_index = (i + 1) * max_chunk_size

        # Obter a parte atual do DataFrame
        df_chunk = dfPrevisao.iloc[start_index:end_index]

        # Converter a parte atual para um dicionário JSON
        json = df_chunk.to_dict(orient='list')
        
        print(json)
        response = requests.post(urlPrevisao, data=json)
        print(response)


In [6]:
def testaExperimento(experimento):
    print('Tratando Experimento: ' , experimento.id)
    global experimentoCorrente
    experimentoCorrente = experimento
    nome_arquivo = '/root/executa-rede/redes/' + experimento.nomeArquivoRede + '.h5'
    
    global modelo
    modelo = tf.keras.models.load_model(nome_arquivo)

    urlExemploTeste = "http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploDadoSinteticoTestes/listaPorAmostraParaTeste?idAmostra={}".format(experimento.amostraDadoSinteticoId)
    print(urlExemploTeste)
    r = requests.get(urlExemploTeste)
    j = r.json()
    
    global Y
    Y = pd.DataFrame.from_dict(j)
    dfTratadoX = Y.apply(trataLinhaX, axis='columns')
    
    global X
    X = np.stack(dfTratadoX.values)
    X.shape
    Y.drop('campoX', inplace=True, axis=1)
    executaPrevisao()
    
    

In [7]:
dfExperimento.apply(testaExperimento, axis='columns')

Tratando Experimento:  1


2023-07-08 17:50:58.191230: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-07-08 17:50:58.202751: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (vps34755): /proc/driver/nvidia/version does not exist
2023-07-08 17:50:58.559261: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


http://vps-40d69db1.vps.ovh.ca:23004/api/ExemploDadoSinteticoTestes/listaPorAmostraParaTeste?idAmostra=1


2023-07-08 17:51:24.051311: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 11520000 exceeds 10% of free system memory.
2023-07-08 17:51:48.449015: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2457600 exceeds 10% of free system memory.
2023-07-08 17:51:48.469385: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2457600 exceeds 10% of free system memory.
2023-07-08 17:51:51.037136: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 2457600 exceeds 10% of free system memory.
2023-07-08 17:51:51.201470: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 1228800 exceeds 10% of free system memory.


300/300 [==============================] - 65s 141ms/step
http://vps-40d69db1.vps.ovh.ca:23004/api/ExperimentoPreRedeResultados/recebePrevisaoTeste
{'id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20], 'score': [0.8213453888893127, 0.821298360824585, 0.8213096857070923, 0.8213412761688232, 0.8213057518005371, 0.8212687373161316, 0.8213351368904114, 0.8212894201278687, 0.8213457465171814, 0.8213329315185547, 0.8212818503379822, 0.8213678002357483, 0.8212965726852417, 0.8212977647781372, 0.8213123679161072, 0.8213375210762024, 0.8213143944740295, 0.8213174343109131, 0.8212909698486328, 0.8212981224060059], 'experimentoId': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']}
<Response [200]>
{'id': [21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40], 'score': [0.8213528990745544, 0.8213464617729187, 0.8213596940040588, 0.8213309049606323, 0.8212701082229614, 0.8213990926742554, 0.82131

...

<Response [200]>
{'id': [8861, 8862, 8863, 8864, 8865, 8866, 8867, 8868, 8869, 8870, 8871, 8872, 8873, 8874, 8875, 8876, 8877, 8878, 8879, 8880], 'score': [0.0011588813504204154, 0.00115891732275486, 0.0011586727341637015, 0.0011587186018005013, 0.0011587637709453702, 0.001158667728304863, 0.001158699276857078, 0.0011588186025619507, 0.0011586827458813787, 0.0011588438646867871, 0.001158936065621674, 0.8213098049163818, 0.001158944913186133, 0.0011588521301746368, 0.001158858765847981, 0.0011587153421714902, 0.0011586126638576388, 0.001158733619377017, 0.0011587964836508036, 0.0011587297776713967], 'experimentoId': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']}
<Response [200]>
{'id': [8881, 8882, 8883, 8884, 8885, 8886, 8887, 8888, 8889, 8890, 8891, 8892, 8893, 8894, 8895, 8896, 8897, 8898, 8899, 8900], 'score': [0.0011588813504204154, 0.0011588174384087324, 0.001158750499598682, 0.8213385939598083, 0.0011589288478717208, 0.001158

<Response [200]>
{'id': [9101, 9102, 9103, 9104, 9105, 9106, 9107, 9108, 9109, 9110, 9111, 9112, 9113, 9114, 9115, 9116, 9117, 9118, 9119, 9120], 'score': [0.8212375044822693, 0.0011587532935664058, 0.001158750499598682, 0.8212602734565735, 0.001158631988801062, 0.0011587454937398434, 0.0011587482877075672, 0.0011587308254092932, 0.0011588813504204154, 0.001158829079940915, 0.0011585387401282787, 0.0011586964828893542, 0.8212270140647888, 0.821457028388977, 0.0011586088221520185, 0.8213234543800354, 0.001158671104349196, 0.0011587042827159166, 0.0011589592322707176, 0.8213450908660889], 'experimentoId': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']}
<Response [200]>
{'id': [9121, 9122, 9123, 9124, 9125, 9126, 9127, 9128, 9129, 9130, 9131, 9132, 9133, 9134, 9135, 9136, 9137, 9138, 9139, 9140], 'score': [0.0011588521301746368, 0.0011586579494178295, 0.8212647438049316, 0.0011588952038437128, 0.0011586324544623494, 0.00115870090667158

<Response [200]>
{'id': [9341, 9342, 9343, 9344, 9345, 9346, 9347, 9348, 9349, 9350, 9351, 9352, 9353, 9354, 9355, 9356, 9357, 9358, 9359, 9360], 'score': [0.8214393854141235, 0.8213191628456116, 0.0011587231419980526, 0.0011587037006393075, 0.0011588599300011992, 0.8212555646896362, 0.0011588234920054674, 0.8212977051734924, 0.0011587798362597823, 0.001158867496997118, 0.8214889764785767, 0.001158800907433033, 0.0011587031185626984, 0.0011587671469897032, 0.0011588736670091748, 0.8213188648223877, 0.0011586230248212814, 0.0011587549233809114, 0.0011588168563321233, 0.0011587203480303288], 'experimentoId': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']}
<Response [200]>
{'id': [9361, 9362, 9363, 9364, 9365, 9366, 9367, 9368, 9369, 9370, 9371, 9372, 9373, 9374, 9375, 9376, 9377, 9378, 9379, 9380], 'score': [0.0011588323395699263, 0.0011588102206587791, 0.0011588422348722816, 0.0011588389752432704, 0.8213703036308289, 0.00115876435302

<Response [200]>
{'id': [9581, 9582, 9583, 9584, 9585, 9586, 9587, 9588, 9589, 9590, 9591, 9592, 9593, 9594, 9595, 9596, 9597, 9598, 9599, 9600], 'score': [0.00115868984721601, 0.0011587981134653091, 0.0011587428161874413, 0.0011588736670091748, 0.0011585927568376064, 0.0011587364133447409, 0.0011586507316678762, 0.0011588218621909618, 0.0011586998589336872, 0.001158671104349196, 0.001158923259936273, 0.0011587621411308646, 0.0011587826302275062, 0.001158646889962256, 0.0011588974157348275, 0.0011585558531805873, 0.0011586633045226336, 0.0011586672626435757, 0.0011587330373004079, 0.001158911851234734], 'experimentoId': ['1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1', '1']}
<Response [200]>


0    None
dtype: object